In [2]:
import numpy as np
import pandas as pd
import pylab
import matplotlib.pyplot as plt
from scipy import stats
import inspect
import os
import csv 
import time
import sys
import glob
import pandas as pd

from tvb.simulator.lab import *
from tvb.simulator.plot.tools import *

# Input Simulation Pipeline
from SimulationPipeline import *

In [11]:
reader = csv.reader(open('do-not-track\\LCycle_G[0.1]_MouseCortex_Params_20200802-201751_.csv'), delimiter=',', quotechar='|')

ParamsDict = {}
for row in reader:
    #print(', '.join(row))
    print(row[0])

BINARY
BOLD
FCFC
G
MODEL
  P [min
  P dtype .................. float64
  P shape .................. (1
  Q [min
  Q dtype .................. float64
  Q shape .................. (1
  Type ..................... WilsonCowan
  a_e [min
  a_e dtype ................ float64
  a_e shape ................ (1
  a_i [min
  a_i dtype ................ float64
  a_i shape ................ (1
  alpha_e [min
  alpha_e dtype ............ float64
  alpha_e shape ............ (1
  alpha_i [min
  alpha_i dtype ............ float64
  alpha_i shape ............ (1
  b_e [min
  b_e dtype ................ float64
  b_e shape ................ (1
  b_i [min
  b_i dtype ................ float64
  b_i shape ................ (1
  c_e [min
  c_e dtype ................ float64
  c_e shape ................ (1
  c_ee [min
  c_ee dtype ............... float64
  c_ee shape ............... (1
  c_ei [min
  c_ei dtype ............... float64
  c_ei shape ............... (1
  c_i [min
  c_i dtype ................ float64


In [8]:
reader = csv.DictReader(open('do-not-track\\LCycle_G[0.1]_MouseCortex_Params_20200802-201751_.csv'))

result = {}
for row in reader:
    print(', '.join(row))

BINARY, True
BINARY, True
BINARY, True
BINARY, True
BINARY, True
BINARY, True
BINARY, True
BINARY, True
BINARY, True
BINARY, True
BINARY, True
BINARY, True
BINARY, True
BINARY, True
BINARY, True
BINARY, True


In [5]:
dictobj

OrderedDict([('BINARY', 'BOLD'), ('True', 'False')])